In [ ]:
import numpy as np
import re

## Day 1

In [ ]:
with open('day1Input.txt') as f:
    data = f.readlines()

numbers = []
for string in data:
    string = string.split("\n")[0]
    # find the first number
    first = re.search('(\d)', string)

    # Find the last (would be the same as first if only one number)
    last = re.search('(\d)', string[::-1])

    numbers.append(first.group(1)+last.group(1))
    
# Make an array of integers and sum
numbers = np.asarray(numbers, dtype=int)
print(numbers.sum())

55621


Now we try to account also for numbers that are written down 

In [ ]:
with open('day1Input.txt') as f:
    data = f.readlines()


# Make a dictionary of all the numbers
numConv = {'one':'1', 'two':'2', 'three':'3', 'four':'4', 'five':'5',
           'six':'6', 'seven':'7', 'eight':'8', 'nine':'9'}




def ReplaceNumbers(string, mapping=numConv):
    '''
    Replaces written numbers with the numerics
    
    INPUT >>>  string = String to check over
               map = Number conversion mapping
              
    OUTPUT >>> string [with all numeric numbers]
    '''
    for word, digit in mapping.items():
        string = re.sub(word, word[0]+digit+word[-1], string, count=0, flags=re.IGNORECASE)
    return string

numbers = []
for string in data:
    string=string.split('\n')[0]
    # convert the numbers to digits
    string = ReplaceNumbers(string)

    # find the first number
    first = re.search('(\d)', string)

    # Find the last (would be the same as first if only one number)
    last = re.search('(\d)', string[::-1])
    numbers.append(first.group(1)+last.group(1))
    
# Make an array of integers and sum
numbers = np.asarray(numbers, dtype=int)


## Day 2

In [33]:
# Dictionary of actual max ball numbers
maxNums = {'red':12,
           'green':13,
           'blue':14}

with open('day2Input.txt') as f:
    data = f.readlines()

# Keep track of legit games
goodGames=[]
for game in data:
    # Find the maximum number of balls shown of each colour in the given game
    redNum = max([int(re.sub(' red', '', string)) for string in re.findall(r'(\d+ red)', game)])
    greenNum = max([int(re.sub(' green', '', string)) for string in re.findall(r'(\d+ green)', game)])
    blueNum = max([int(re.sub(' blue', '', string)) for string in re.findall(r'(\d+ blue)', game)])

    # If all of these maximums are less than the max allowed then the game is legit and store the game id
    if (redNum<=maxNums['red'])&(greenNum<=maxNums['green'])&(blueNum<=maxNums['blue']):
        goodGames.append(int(re.search('(Game \d+)', game).group(1).replace('Game ', '')))

print(f'{len(goodGames)} possible legit games with summed ids to {sum(goodGames)}')


46 possible legit games with summed ids to 2283


Now we find the fewest number of cubes of each colour that could have made the game possible

In [34]:
with open('day2Input.txt') as f:
    data = f.readlines()

cubePower=[]
for game in data:
    # Find the maximum number of balls shown of each colour in the given game
    # This max number is also the fewest needed of that colour
    redNum = max([int(re.sub(' red', '', string)) for string in re.findall(r'(\d+ red)', game)])
    greenNum = max([int(re.sub(' green', '', string)) for string in re.findall(r'(\d+ green)', game)])
    blueNum = max([int(re.sub(' blue', '', string)) for string in re.findall(r'(\d+ blue)', game)])

    # Append the product of these values
    cubePower.append(redNum*greenNum*blueNum)

print(f'Sum of cube power is {sum(cubePower)}')
    

Sum of cube power is 78669
